In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [33]:
import warnings

import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    fbeta_score,
    f1_score,
    make_scorer,
    accuracy_score,
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [4]:
feat_30_loc = "../data/features_30_sec.csv"
feat_3_loc = "../data/features_3_sec.csv"
# named long and short to differentiate easier
long = pd.read_csv(feat_30_loc)
short = pd.read_csv(feat_3_loc)

long_og = long.copy()
short_og = short.copy()

<IPython.core.display.Javascript object>

In [5]:
var_cols = short.columns[short.columns.str.contains("_var")]
logged_var_long_df = long.copy()
logged_var_short_df = short.copy()
for col in var_cols:
    logged_var_long_df[col + "_logged"] = np.log(logged_var_long_df[col])
    logged_var_long_df = logged_var_long_df.drop(col, 1)
    logged_var_short_df[col + "_logged"] = np.log(logged_var_short_df[col])
    logged_var_short_df = logged_var_short_df.drop(col, 1)

<IPython.core.display.Javascript object>

In [6]:
long_X = logged_var_long_df.drop("label", 1)
long_y = logged_var_long_df["label"]

long_X_train, long_X_test, long_y_train, long_y_test = train_test_split(
    long_X, long_y, test_size=0.2, random_state=34, stratify=long_y
)

<IPython.core.display.Javascript object>

In [7]:
files_stripped = long_X_train["filename"].str.strip(".wav")
short_train_df = logged_var_short_df[
    logged_var_short_df["filename"].str.slice(0, -6).isin(list(files_stripped))
]
short_train_df

,filename,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,...,mfcc11_var_logged,mfcc12_var_logged,mfcc13_var_logged,mfcc14_var_logged,mfcc15_var_logged,mfcc16_var_logged,mfcc17_var_logged,mfcc18_var_logged,mfcc19_var_logged,mfcc20_var_logged
0,blues.00000.0.wav,66149,0.335406,0.130405,1773.065032,1972.744388,3714.560359,0.081851,-0.000078,-6.816183e-05,...,4.199753,3.808962,3.531333,3.876799,3.755859,3.681027,3.596990,3.640192,3.515064,3.778989
1,blues.00000.1.wav,66149,0.343065,0.112699,1816.693777,2010.051501,3869.682242,0.087173,-0.000099,-1.033452e-04,...,3.995884,3.705316,3.427791,3.887407,4.228071,4.170507,3.705679,3.936991,4.575029,4.093395
2,blues.00000.2.wav,66149,0.346815,0.132003,1788.539719,2084.565132,3997.639160,0.071383,-0.000066,6.505577e-05,...,3.869358,3.599226,3.706844,3.423834,3.918273,4.209703,3.344577,3.822484,3.971251,3.499688
3,blues.00000.3.wav,66149,0.363639,0.132565,1655.289045,1960.039988,3568.300218,0.069426,-0.000014,1.825280e-05,...,4.069718,3.666174,4.106187,3.699115,3.799220,3.865758,3.344173,3.548782,3.928609,3.466476
4,blues.00000.4.wav,66149,0.335579,0.143289,1630.656199,1948.503884,3469.992864,0.070095,0.000041,-1.059828e-04,...,4.007883,3.766533,3.839780,3.217123,3.684292,3.412347,3.826049,3.938929,3.286115,3.372341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,rock.00098.5.wav,66149,0.423990,0.105415,1603.248807,1832.425252,3275.200195,0.072465,-0.000043,-5.109625e-05,...,3.695100,3.619010,3.516421,3.100012,3.826093,3.883990,3.567032,3.426182,3.394696,3.360307
9976,rock.00098.6.wav,66149,0.365676,0.109754,1522.404166,1799.056171,3192.628643,0.062293,0.000025,1.812802e-07,...,3.498222,3.638079,3.209702,3.761270,3.783397,3.759324,3.771833,3.663633,2.980535,3.562415
9977,rock.00098.7.wav,66149,0.328398,0.056352,1382.797488,1811.470996,3016.222018,0.045245,-0.000001,2.240326e-05,...,4.057362,4.974400,4.457256,3.593842,4.188851,4.331148,3.875327,3.697891,3.690622,4.037018
9978,rock.00098.8.wav,66149,0.254980,0.018005,1003.694252,1581.533448,1956.539964,0.022277,-0.000014,-2.279821e-06,...,3.747703,4.004866,3.609216,3.817424,3.675378,3.659123,2.780470,3.795206,3.364819,3.147167


<IPython.core.display.Javascript object>

In [8]:
files_stripped = long_X_test["filename"].str.strip(".wav")
short_test_df = logged_var_short_df[
    logged_var_short_df["filename"].str.slice(0, -6).isin(list(files_stripped))
]
short_test_df

,filename,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,...,mfcc11_var_logged,mfcc12_var_logged,mfcc13_var_logged,mfcc14_var_logged,mfcc15_var_logged,mfcc16_var_logged,mfcc17_var_logged,mfcc18_var_logged,mfcc19_var_logged,mfcc20_var_logged
50,blues.00005.0.wav,66149,0.302958,0.074403,1148.336755,1481.481546,2285.749512,0.049448,-0.000021,-0.000021,...,3.129887,4.317805,3.868644,3.198746,3.220440,3.638702,2.955164,3.163215,3.462902,3.156687
51,blues.00005.1.wav,66149,0.325506,0.080293,1210.571572,1424.920044,2228.934983,0.057752,0.000025,-0.000009,...,3.874508,3.669179,3.952440,3.325389,2.593266,3.462044,3.741462,3.500137,2.721179,3.418986
52,blues.00005.2.wav,66149,0.299864,0.156715,3191.200301,2190.240135,5734.540452,0.190655,-0.000002,-0.000026,...,4.811318,4.949066,4.504757,4.642990,5.082304,4.432605,4.186323,4.351435,3.565979,4.315627
53,blues.00005.3.wav,66149,0.299961,0.124830,2124.493966,1771.906237,3852.538499,0.121161,0.000025,-0.000017,...,3.871562,4.349438,4.354326,3.760185,4.023502,4.525363,4.240223,4.191362,3.508737,3.779347
54,blues.00005.4.wav,66149,0.323245,0.073035,2446.168018,2092.543627,4750.638709,0.114742,-0.000025,-0.000051,...,5.086935,4.709287,4.623443,3.945663,5.041515,4.050801,4.139557,3.832766,3.993757,3.859010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.050019,1499.083005,1718.707215,3015.559458,0.072885,-0.000034,-0.000020,...,3.416803,3.839206,3.935643,3.410298,4.386414,3.749174,3.646150,3.435594,3.887814,3.662714
9986,rock.00099.6.wav,66149,0.372564,0.057897,1847.965128,1906.468492,3746.694524,0.089111,-0.000027,-0.000045,...,3.501858,3.690007,3.688708,3.818595,3.533665,3.478628,4.195977,3.996626,4.150288,2.928365
9987,rock.00099.7.wav,66149,0.347481,0.052403,1346.157659,1561.859087,2442.362154,0.072194,0.000015,-0.000029,...,4.520072,3.518154,3.361223,4.594477,3.787080,4.359630,3.080946,3.257359,3.883261,3.727787
9988,rock.00099.8.wav,66149,0.387527,0.066430,2084.515327,2018.366254,4313.266226,0.104072,0.000004,-0.000048,...,3.353508,3.200891,2.922559,2.939444,3.026968,3.343700,2.845487,3.065395,3.212601,2.548488


<IPython.core.display.Javascript object>

In [37]:
m_start = 18  # highest mfcc to use. higher than this is too high in the frequency spectrum to really matter
mel_freq_drops = [f"mfcc{x}_mean" for x in range(m_start, 21)] + [
    f"mfcc{x}_var_logged" for x in range(m_start, 21)
]

<IPython.core.display.Javascript object>

In [41]:
# best balance for VIF I could tune
drop_cols = [
    "length",
    "filename",
    "label",
    #     "zero_crossing_rate_mean",  # and this
    #     "zero_crossing_rate_var_logged",  # so does this
    "rolloff_mean",  # and this
    #     "harmony_var_logged",  # and this
    "rolloff_var_logged",
    "spectral_centroid_var_logged",
    #     "spectral_bandwidth_var_logged",
    "spectral_centroid_mean",
    "spectral_bandwidth_mean",
    "rms_mean",
    "rms_var_logged",
    #     "mfcc1_mean",
    #     "mfcc2_mean",
    "perceptr_var_logged",
    #     "chroma_stft_mean",
    #     "chroma_stft_var_logged",
    #     "harmony_mean",  # This causes high training guessing
    #     "perceptr_mean",  # this one
    #     "tempo",
]
drop_cols = drop_cols + mel_freq_drops
print_vif(logged_var_short_df.drop(drop_cols, 1,))

VIF results
-------------------------------
const                            1853.345520
chroma_stft_mean                    3.834928
zero_crossing_rate_mean             6.432165
harmony_mean                        1.486140
perceptr_mean                       1.587434
tempo                               1.010578
mfcc1_mean                          9.629508
mfcc2_mean                          6.428282
mfcc3_mean                          2.562475
mfcc4_mean                          2.378938
mfcc5_mean                          2.879489
mfcc6_mean                          3.475823
mfcc7_mean                          3.203901
mfcc8_mean                          4.048924
mfcc9_mean                          2.809137
mfcc10_mean                         2.974507
mfcc11_mean                         2.420472
mfcc12_mean                         2.837060
mfcc13_mean                         2.648171
mfcc14_mean                         2.314453
mfcc15_mean                         2.492834
mfcc16_mean

<IPython.core.display.Javascript object>

In [42]:
X_train = short_train_df.drop(drop_cols, 1)
X_test = short_test_df.drop(drop_cols, 1)
y_train = short_train_df["label"]
y_test = short_test_df["label"]

<IPython.core.display.Javascript object>

In [43]:
num_cols = list(X_train.columns)

bin_cols = []

cat_cols = []
drop_cats = []


preprocessing = ColumnTransformer(
    [
        # Scale numeric columns (not needed for all models but can't hurt)
        ("scaler", StandardScaler(), num_cols)
    ],
    remainder="passthrough",
)


pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        #         ("pca", PCA(n_components=6)),
        # Choose your model and put it here
        ("svc", SVC()),
    ]
)


params = {
    # Use model__ with hyperprammeter names after
    "svc__C": [1, 10, 100],
    "svc__kernel": ["linear"],
    "svc__decision_function_shape": ["ovo", "ovr"],
    #     "svc__degree": [2, 3, 5],
}


pipeline_cv = GridSearchCV(pipeline, params, verbose=1, n_jobs=-1, cv=2)

pipeline_cv.fit(X_train, y=y_train)


print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))
pipeline_cv.best_params_

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   26.5s finished


0.7677096370463079
0.6605


{'svc__C': 10, 'svc__decision_function_shape': 'ovo', 'svc__kernel': 'linear'}

<IPython.core.display.Javascript object>

In [ ]:
num_cols = list(X_train.columns)

bin_cols = []

cat_cols = []
drop_cats = []


preprocessing = ColumnTransformer(
    [
        # Scale numeric columns (not needed for all models but can't hurt)
        ("scaler", StandardScaler(), num_cols)
    ],
    remainder="passthrough",
)


pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        #         ("pca", PCA()),
        # Choose your model and put it here
        ("knn", KNeighborsClassifier()),
    ]
)


params = {
    "knn__n_neighbors": [ 5,10,15],
    "knn__weights": ["distance"],
    "knn__leaf_size": [15, 30],
    "knn__algorithm": ["kd_tree", "ball_tree"],
}


pipeline_cv = GridSearchCV(pipeline, params, verbose=1, n_jobs=-1, cv=2)

pipeline_cv.fit(X_train, y=y_train)


print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))
pipeline_cv.best_params_

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    7.7s finished


all_feat_cols = list(short_train_df.columns[3:])
all_feat_cols.remove("label")

for col in all_feat_cols:
    X_train = short_train_df[[col]]
    X_test = short_test_df[[col]]
    y_train = short_train_df["label"]
    y_test = short_test_df["label"]
    #     X_train, X_test, y_train, y_test = train_test_split(
    #         X, y, test_size=0.2, random_state=34, stratify=y
    #     )
    num_cols = [col]

    bin_cols = []

    cat_cols = []
    drop_cats = []

    preprocessing = ColumnTransformer(
        [
            # Scale numeric columns (not needed for all models but can't hurt)
            ("scaler", StandardScaler(), num_cols)
        ],
        remainder="passthrough",
    )

    pipeline = Pipeline(
        [
            ("preprocessing", preprocessing),
            #         ("pca", PCA()),
            # Choose your model and put it here
            # ("knn", KNeighborsClassifier()),
            # ("svc", SVC()),
            #             ("rfc", RandomForestClassifier()),
            ("xgb", XGBClassifier()),
        ]
    )

    # params = {
    #         "knn__n_neighbors": [5],
    #         "knn__weights": ["distance"],
    #         "knn__leaf_size": [30],
    #         "knn__algorithm": ["kd_tree"],
    #     }
    # pipeline_cv = GridSearchCV(pipeline, params, verbose=0, n_jobs=-1, cv=2)

    pipeline.fit(X_train, y=y_train)

    print(col)
    print(pipeline.score(X_train, y_train))
    print(pipeline.score(X_test, y_test))

feature_importance = pipeline_cv.best_estimator_.named_steps["xgb"].feature_importances_

# Make importances relative to max importance.
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5


plt.figure(figsize=(15, 20))
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel("Relative Importance")
plt.title("Variable Importance")
plt.show()

In [13]:
y_pred = pipeline_cv.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[159   1  11   0   0   8   5   0  11   5]
 [  0 195   0   0   0   5   0   0   0   0]
 [ 26   1 115  11   6  13   4   4   3  17]
 [  1   0   4 149  13   0   3  16   2  12]
 [ 16   0   2  13 109   0  12  24  22   2]
 [  2  33  10   1   0 140   0   1   5   8]
 [  3   0   1  27   3   0 157   0   2   7]
 [  0   1  17   6  13   2   0 151   3   7]
 [ 10   0  20  10  17   2   7  12 116   6]
 [ 14   2  26  42   4  20  16  18   6  52]]
              precision    recall  f1-score   support

       blues       0.69      0.80      0.74       200
   classical       0.84      0.97      0.90       200
     country       0.56      0.57      0.57       200
       disco       0.58      0.74      0.65       200
      hiphop       0.66      0.55      0.60       200
        jazz       0.74      0.70      0.72       200
       metal       0.77      0.79      0.78       200
         pop       0.67      0.76      0.71       200
      reggae       0.68      0.58      0.63       200
        rock       0.45     

<IPython.core.display.Javascript object>

In [15]:
short["predicted"] = pipeline_cv.predict(logged_var_short_df.drop(drop_cols, 1))

<IPython.core.display.Javascript object>

In [16]:
long["vote_pred"] = "none"

for i in range(long["filename"].size):
    curr_file = long["filename"][i]
    file_stripped = curr_file.strip(".wav")
    sub_selection = short["filename"].str.contains(file_stripped)
    prediction = (
        short[sub_selection]["predicted"]
        .value_counts()
        .sort_values(ascending=False)
        .index[0]
    )
    long["vote_pred"][i] = prediction

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


<IPython.core.display.Javascript object>

In [17]:
print(confusion_matrix(long["label"], long["vote_pred"]))
print(classification_report(long["label"], long["vote_pred"]))

[[ 96   1   1   0   0   0   0   0   1   1]
 [  0 100   0   0   0   0   0   0   0   0]
 [  8   0  79   3   0   2   0   1   0   7]
 [  0   0   3  79   6   0   3   3   2   4]
 [  4   0   2   3  72   1   3   7   8   0]
 [  2   6   1   0   0  90   0   0   0   1]
 [  1   0   0   3   0   0  94   0   0   2]
 [  0   0   4   2   4   0   0  85   4   1]
 [  3   0   2   4   8   1   1   6  74   1]
 [  9   0   6  15   1   5   4   6   3  51]]
              precision    recall  f1-score   support

       blues       0.78      0.96      0.86       100
   classical       0.93      1.00      0.97       100
     country       0.81      0.79      0.80       100
       disco       0.72      0.79      0.76       100
      hiphop       0.79      0.72      0.75       100
        jazz       0.91      0.90      0.90       100
       metal       0.90      0.94      0.92       100
         pop       0.79      0.85      0.82       100
      reggae       0.80      0.74      0.77       100
        rock       0.75     

<IPython.core.display.Javascript object>

In [19]:
long_pred_df = logged_var_long_df.drop(drop_cols, 1)
# long_pred_df = scaler.transform(long_pred_df)
long["model_pred"] = pipeline_cv.predict(long_pred_df)

<IPython.core.display.Javascript object>

In [20]:
print(confusion_matrix(long["label"], long["model_pred"]))
print(classification_report(long["label"], long["model_pred"]))

[[87  0  5  1  0  0  1  0  6  0]
 [ 0 96  0  0  0  3  0  0  1  0]
 [ 6  0 73  5  0  1  0  2  7  6]
 [ 1  0  3 72  8  0  2  5  6  3]
 [ 3  0  1  6 74  0  2  5  9  0]
 [ 7  4  5  0  0 82  0  0  0  2]
 [ 4  0  0  5  4  0 86  0  0  1]
 [ 0  0  2  1  5  0  0 85  6  1]
 [ 0  0  4  2  6  0  1  5 82  0]
 [12  0  7 12  2  2  3  7 12 43]]
              precision    recall  f1-score   support

       blues       0.72      0.87      0.79       100
   classical       0.96      0.96      0.96       100
     country       0.73      0.73      0.73       100
       disco       0.69      0.72      0.71       100
      hiphop       0.75      0.74      0.74       100
        jazz       0.93      0.82      0.87       100
       metal       0.91      0.86      0.88       100
         pop       0.78      0.85      0.81       100
      reggae       0.64      0.82      0.72       100
        rock       0.77      0.43      0.55       100

    accuracy                           0.78      1000
   macro avg       

<IPython.core.display.Javascript object>

In [21]:
drop_cols_copy = drop_cols.copy()
drop_cols_copy.remove("label")
long_y_pred = pipeline_cv.predict(long_X_test.drop(drop_cols_copy, 1))

<IPython.core.display.Javascript object>

In [22]:
# model prediction on original train test split on the long df training
pipeline_cv.score(long_X_train.drop(drop_cols_copy, 1), long_y_train)

0.79875

<IPython.core.display.Javascript object>

In [23]:
# model prediction on original train test split on the long df testing
pipeline_cv.score(long_X_test.drop(drop_cols_copy, 1), long_y_test)

0.705

<IPython.core.display.Javascript object>

In [24]:
long_y_pred
print(confusion_matrix(long_y_test, long_y_pred))
print(classification_report(long_y_test, long_y_pred))

[[16  0  1  0  0  0  1  0  2  0]
 [ 0 19  0  0  0  1  0  0  0  0]
 [ 2  0 14  2  0  1  0  0  0  1]
 [ 0  0  0 16  1  0  0  1  1  1]
 [ 1  0  0  2 12  0  1  2  2  0]
 [ 1  2  2  0  0 15  0  0  0  0]
 [ 0  0  0  4  1  0 15  0  0  0]
 [ 0  0  1  0  3  0  0 15  0  1]
 [ 0  0  3  0  2  0  1  1 13  0]
 [ 1  0  3  5  0  1  0  3  1  6]]
              precision    recall  f1-score   support

       blues       0.76      0.80      0.78        20
   classical       0.90      0.95      0.93        20
     country       0.58      0.70      0.64        20
       disco       0.55      0.80      0.65        20
      hiphop       0.63      0.60      0.62        20
        jazz       0.83      0.75      0.79        20
       metal       0.83      0.75      0.79        20
         pop       0.68      0.75      0.71        20
      reggae       0.68      0.65      0.67        20
        rock       0.67      0.30      0.41        20

    accuracy                           0.70       200
   macro avg       

<IPython.core.display.Javascript object>

In [25]:
long_vote_pred = long["vote_pred"]
long_label = long["label"]

(
    long_vote_pred_train,
    long_vote_pred_test,
    long_label_train,
    long_label_test,
) = train_test_split(
    long_vote_pred, long_label, test_size=0.2, random_state=34, stratify=long_label
)

<IPython.core.display.Javascript object>

In [26]:
print(confusion_matrix(long_label_test, long_vote_pred_test))
print(classification_report(long_label_test, long_vote_pred_test))
print(accuracy_score(long_label_test, long_vote_pred_test))

[[18  0  1  0  0  0  0  0  1  0]
 [ 0 20  0  0  0  0  0  0  0  0]
 [ 2  0 15  1  0  1  0  0  0  1]
 [ 0  0  0 17  1  0  0  1  0  1]
 [ 1  0  0  1 12  0  2  2  2  0]
 [ 0  3  1  0  0 16  0  0  0  0]
 [ 0  0  0  3  0  0 17  0  0  0]
 [ 0  0  2  0  2  0  0 15  0  1]
 [ 1  0  1  0  1  0  1  2 13  1]
 [ 1  0  3  6  0  2  1  3  1  3]]
              precision    recall  f1-score   support

       blues       0.78      0.90      0.84        20
   classical       0.87      1.00      0.93        20
     country       0.65      0.75      0.70        20
       disco       0.61      0.85      0.71        20
      hiphop       0.75      0.60      0.67        20
        jazz       0.84      0.80      0.82        20
       metal       0.81      0.85      0.83        20
         pop       0.65      0.75      0.70        20
      reggae       0.76      0.65      0.70        20
        rock       0.43      0.15      0.22        20

    accuracy                           0.73       200
   macro avg       

<IPython.core.display.Javascript object>

In [27]:
# majority vote predictions on training data
print(confusion_matrix(long_label_train, long_vote_pred_train))
print(classification_report(long_label_train, long_vote_pred_train))
print(accuracy_score(long_label_train, long_vote_pred_train))

[[78  1  0  0  0  0  0  0  0  1]
 [ 0 80  0  0  0  0  0  0  0  0]
 [ 6  0 64  2  0  1  0  1  0  6]
 [ 0  0  3 62  5  0  3  2  2  3]
 [ 3  0  2  2 60  1  1  5  6  0]
 [ 2  3  0  0  0 74  0  0  0  1]
 [ 1  0  0  0  0  0 77  0  0  2]
 [ 0  0  2  2  2  0  0 70  4  0]
 [ 2  0  1  4  7  1  0  4 61  0]
 [ 8  0  3  9  1  3  3  3  2 48]]
              precision    recall  f1-score   support

       blues       0.78      0.97      0.87        80
   classical       0.95      1.00      0.98        80
     country       0.85      0.80      0.83        80
       disco       0.77      0.78      0.77        80
      hiphop       0.80      0.75      0.77        80
        jazz       0.93      0.93      0.93        80
       metal       0.92      0.96      0.94        80
         pop       0.82      0.88      0.85        80
      reggae       0.81      0.76      0.79        80
        rock       0.79      0.60      0.68        80

    accuracy                           0.84       800
   macro avg       

<IPython.core.display.Javascript object>